In [ ]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid
import pickle
import dataEngr
reload(dataEngr)

Read in just the testing and training data.

In [4]:
xgbInput = dataEngr.clfInput()
xgbInput.get_sessionsFtr()
xgbInput.users_ftrEng()
xgbInput.one_hot()
#xgbInput.binarize_targets()
xgbInput.split_data(update_trainDf=True)

number of columns before one hot encoding 370
number of columns after one hot encoding 469


In [ ]:
condition = 'dac_year == 2014 & action_counts != -1'
len(xgbInput.trainDf.query(condition))

In [1]:
%run -i run_model.py

number of columns before one hot encoding 370
number of columns after one hot encoding 469


In [ ]:
def cv_sessions(xgbInput):
    """Cross-validation on a subset of training data which meets a condition"""
    
    condition = 'dac_year == 2014 & action_counts != -1'
    idx = np.nonzero(xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index))
    idx = idx[0]
    np.random.shuffle(idx[0])
    n = len(idx)
    
    for i in range(0, kfolds):
        idx[0]
        
def cv_bydate(xgbInput):
    """Select folds for cross validation as all cases that occurred in a given with month 
    in 2014, with sessions data.
    Only cases in 2014 have sessions data and the last test case is on June 30.
    """
    
    for i in range(4,7):
        condition = 'dac_year == 2014 & dac_month == @i & action_counts != -1'
        valid_mask = xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index)
        valid_indx = np.where(valid_mask)[0]
        train_indx = np.where(~valid_mask)[0]
        yield train_indx, valid_indx

In [ ]:
pd.read_pickle('cv_results/sessions_e20_25n/validate_err.p')

In [ ]:
a = pd.read_pickle('cv_results/actions_e20/validate_err.p')
a = a.astype(float)
a['mean'] = a.mean(axis = 1)

In [ ]:
%run run_model.py

In [5]:
with open('../xgbmodels/actions2_e16_53n.p') as f:
    bst = pickle.load(f)

In [ ]:
xgbInput.allDf.columns[xgbInput.allDf.columns.str.contains('book')]

In [7]:
len(features)

340

In [9]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, xgbInput.allDf.columns[int(feat[1:])], score) for feat, score in features]

[('f351', 'age', 1614),
 ('f359', 'tfa_hour', 1214),
 ('f354', 'dac_day', 1109),
 ('f322', 'secs_elapsed', 808),
 ('f353', 'dac_month', 760),
 ('f355', 'dac_dayofweek', 620),
 ('f358', 'tfa_day', 553),
 ('f321', 'action_counts', 551),
 ('f357', 'tfa_month', 483),
 ('f352', 'dac_year', 402),
 ('f329', 'Mac Desktop_counts', 333),
 ('f233', 'requested', 328),
 ('f254', 'show_p3', 308),
 ('f27', 'ajax_refresh_subtotal', 298),
 ('f136', 'index_view_search_results', 295),
 ('f244', 'search_results', 294),
 ('f364', 'signup_method_basic', 259),
 ('f266', 'similar_listings', 258),
 ('f360', 'gender_-unknown-', 252),
 ('f190', 'personalize', 242),
 ('f156', 'lookup', 234),
 ('f427', 'first_affiliate_tracked_untracked', 220),
 ('f362', 'gender_MALE', 215),
 ('f306', 'update_update_listing', 215),
 ('f356', 'tfa_year', 212),
 ('f332', 'Windows Desktop_counts', 207),
 ('f188', 'pending_pending', 198),
 ('f435', 'first_device_type_Mac Desktop', 192),
 ('f361', 'gender_FEMALE', 189),
 ('f257', 'show